In [1]:
from tardis.io.config_reader import Configuration
from tardis.simulation import Simulation
from tardis.plasma.standard_plasmas import assemble_plasma
from tardis.model import Radial1DModel
from tardis.io.atom_data import AtomData
from tardis.analysis import LastLineInteraction
from tardis.util.base import int_to_roman
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from tardis.io.config_internal import get_data_dir
from astropy import units as u
import plotly.io as pio
pio.renderers.default='notebook'
import os

Iterations:          0/? [00:00<?, ?it/s]

Packets:             0/? [00:00<?, ?it/s]

In [2]:
### Run Simulation
config = Configuration.from_yaml('../../../docs/tardis_example.yml')
atom_data = AtomData.from_hdf(os.path.join(get_data_dir(), 'kurucz_cd23_chianti_H_He.h5'))
model = Radial1DModel.from_config(config, atom_data=atom_data)
plasma = assemble_plasma(config, model, atom_data=atom_data)
sim = Simulation.from_config(config, model=model, plasma=plasma)
sim.run()

Abundances have not been normalized to 1. - normalizing
Zeta_data missing - replaced with 1s. Missing ions: [(12, 13), (14, 15), (16, 17), (18, 19), (20, 21)]
/Users/archil/Documents/tardis_ayushi/tardis/plasma/properties/radiative_properties.py:93: RuntimeWarning:

divide by zero encountered in true_divide

/Users/archil/Documents/tardis_ayushi/tardis/plasma/properties/radiative_properties.py:93: RuntimeWarning:

invalid value encountered in true_divide

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
Zeta_data missing - replaced with 1s. Missing ions: [(12, 13), (14, 15), (16, 17), (18, 19), (20, 21)]
/Users/archil/Documents/tardis_ayushi/tardis/plasma/properties/radiative_properties.py:93: RuntimeWarning:

divide by zero encountered in true_divide

/Users/archil/Documents/tardis_ayushi/tardis/plasma/properties/radiative_properties.py:93: RuntimeWarning:

invalid value encountered in true_divide



TqdmHBox(children=(HTML(value='Iterations:', layout=Layout(width='6%')), FloatProgress(value=0.0, layout=Layou…

TqdmHBox(children=(HTML(value='Packets:   ', layout=Layout(width='6%')), FloatProgress(value=0.0, layout=Layou…

Shell No.,t_rad,next_t_rad,w,next_w
0,9.93e+03,1.01e+04,0.4,0.525
5,9.85e+03,1.03e+04,0.211,0.196
10,9.78e+03,1.02e+04,0.143,0.115
15,9.71e+03,9.88e+03,0.105,0.0843


/Users/archil/Documents/tardis_ayushi/tardis/plasma/properties/radiative_properties.py:93: RuntimeWarning:

divide by zero encountered in true_divide

/Users/archil/Documents/tardis_ayushi/tardis/plasma/properties/radiative_properties.py:93: RuntimeWarning:

invalid value encountered in true_divide



...

Shell No.,t_rad,next_t_rad,w,next_w
0,1.12e+04,1.12e+04,0.48,0.486
5,1.16e+04,1.15e+04,0.174,0.18
10,1.13e+04,1.12e+04,0.105,0.108
15,1.09e+04,1.09e+04,0.0789,0.0793


Shell No.,t_rad,next_t_rad,w,next_w
0,1.12e+04,1.12e+04,0.486,0.486
5,1.15e+04,1.15e+04,0.18,0.177
10,1.12e+04,1.13e+04,0.108,0.107
15,1.09e+04,1.09e+04,0.0793,0.0811


Shell No.,t_rad,next_t_rad,w,next_w
0,1.12e+04,1.12e+04,0.486,0.483
5,1.15e+04,1.16e+04,0.177,0.17
10,1.13e+04,1.13e+04,0.107,0.107
15,1.09e+04,1.09e+04,0.0811,0.0799


Shell No.,t_rad,next_t_rad,w,next_w
0,1.12e+04,1.12e+04,0.483,0.482
5,1.16e+04,1.16e+04,0.17,0.172
10,1.13e+04,1.13e+04,0.107,0.105
15,1.09e+04,1.09e+04,0.0799,0.0807


Shell No.,t_rad,next_t_rad,w,next_w
0,1.12e+04,1.12e+04,0.482,0.478
5,1.16e+04,1.14e+04,0.172,0.177
10,1.13e+04,1.13e+04,0.105,0.107
15,1.09e+04,1.08e+04,0.0807,0.0814


In [234]:
class GrotrianWidget:
    def __init__(self, sim, atomic_number, ion_number,
                 num_shells=10, level_diff_threshold=0.01, filter_mode='packet_out_nu'):
        self.atomic_number = atomic_number
        self.atom_name = sim.plasma.atomic_data.atom_data.loc[atomic_number]['name']
        self.atomic_symbol = sim.plasma.atomic_data.atom_data.loc[atomic_number]['symbol']
        self.ion_number = ion_number
        
        self.num_shells = num_shells
        self.level_diff_threshold = level_diff_threshold
        self.filter_mode = filter_mode
        
        self.compute_level_data(sim)
        
        self.compute_transitions(sim)
        
        self.level_width_scale, self.level_width_offset = 3, 1
        self.transition_width_scale, self.transition_width_offset = 2, 1
        
        # Define colors for the transitions based on wavelengths
        self.colorscale = 'plasma'
        self.cmap = plt.get_cmap(self.colorscale)
        self.min_wavelength = sim.plasma.lines.wavelength.min()
        self.max_wavelength = sim.plasma.lines.wavelength.max()
    
    def compute_transitions(self, sim):
        ### Get lines data
        lines_data = sim.plasma.atomic_data.lines.loc[self.atomic_number, self.ion_number].reset_index().set_index('line_id')

        # Get relevant lines for current simulation
        last_line_interaction = LastLineInteraction.from_model(sim, self.filter_mode)
        last_line_interaction.atomic_number = self.atomic_number
        last_line_interaction.ion_number = self.ion_number
        
        # Get the excitation/de-excitation transitions from last_line_interaction object
        excit_lines = last_line_interaction.last_line_in.reset_index().groupby(['level_number_lower', 'level_number_upper']).agg(
            num_electrons=('line_id', 'count'), # Take count of lines
            wavelength=('wavelength', 'first'), # Take first of wavelengths
        ).reset_index()

        deexcit_lines = last_line_interaction.last_line_out.reset_index().groupby(['level_number_lower', 'level_number_upper']).agg(
            num_electrons=('line_id', 'count'), # Take count of lines
            wavelength=('wavelength', 'first'), # Take first of wavelengths
        ).reset_index()

        # Filter transitions to only include transitions upto the NUM_SHELLS
        excit_lines = excit_lines.loc[
            (excit_lines.level_number_lower <= NUM_SHELLS) & (excit_lines.level_number_upper <= NUM_SHELLS)
        ]
        deexcit_lines = deexcit_lines.loc[
            (deexcit_lines.level_number_lower <= NUM_SHELLS) & (deexcit_lines.level_number_upper <= NUM_SHELLS)
        ]

        # Map the levels to merged levels
        excit_lines['merged_level_number_lower'] = excit_lines.level_number_lower.map(self.level_mapping)
        excit_lines['merged_level_number_upper'] = excit_lines.level_number_upper.map(self.level_mapping)
        deexcit_lines['merged_level_number_lower'] = deexcit_lines.level_number_lower.map(self.level_mapping)
        deexcit_lines['merged_level_number_upper'] = deexcit_lines.level_number_upper.map(self.level_mapping)

        # Group by level pairs
        excit_lines = excit_lines.groupby(['merged_level_number_lower', 'merged_level_number_upper']).agg(
            wavelength=('wavelength', 'mean'), # Take mean of wavelength
            num_electrons=('num_electrons', 'sum'), # Take sum of counts
        ).reset_index()
        deexcit_lines = deexcit_lines.groupby(['merged_level_number_lower', 'merged_level_number_upper']).agg(
            wavelength=('wavelength', 'mean'), # Take mean of wavelength
            num_electrons=('num_electrons', 'sum'), # Take sum of counts
        ).reset_index()

        # Remove the rows where start and end (merged) level is the same
        excit_lines = excit_lines.loc[excit_lines.merged_level_number_lower != excit_lines.merged_level_number_upper]
        deexcit_lines = deexcit_lines.loc[deexcit_lines.merged_level_number_lower != deexcit_lines.merged_level_number_upper]

        # Compute the standardized log number of electrons for transition line thickness and offset
        excit_log_num_electrons_range = np.log10(np.max(excit_lines.num_electrons)) - np.log10(np.min(excit_lines.num_electrons))
        excit_lines['standard_log_num_electrons'] = (np.log10(excit_lines.num_electrons) - np.log10(np.min(excit_lines.num_electrons))) / excit_log_num_electrons_range

        deexcit_log_num_electrons_range = np.log10(np.max(deexcit_lines.num_electrons)) - np.log10(np.min(deexcit_lines.num_electrons))
        deexcit_lines['standard_log_num_electrons'] = (np.log10(deexcit_lines.num_electrons) - np.log10(np.min(deexcit_lines.num_electrons))) / deexcit_log_num_electrons_range
        
        self.excit_lines = excit_lines
        self.deexcit_lines = deexcit_lines

    
    def compute_level_data(self, sim):
        ### Get energy levels and convert to eV
        raw_energy_levels = sim.plasma.atomic_data.levels.loc[self.atomic_number, self.ion_number].loc[1:self.num_shells]
        raw_energy_levels = raw_energy_levels.energy / (u.electronvolt.to('J') * 1e7)
        raw_energy_levels = pd.Series(raw_energy_levels, name='energy')
        
        ### Get level populations
        raw_level_populations = sim.plasma.level_number_density.loc[self.atomic_number, self.ion_number].loc[1:self.num_shells]

        # Average out the level populations across all zones (TODO: might include an option to select the zone number later)
        raw_level_populations = raw_level_populations.mean(axis=1)

        raw_level_populations = pd.Series(raw_level_populations, name='population')
        
        ### Join level populations and energy values
        raw_level_data = pd.merge(raw_energy_levels, raw_level_populations, left_index=True, right_index=True)
        
        ### Merge the levels if energy difference is too less
        # Get new level numbers
        # TODO: Find a better way to find close levels
        raw_level_data['merged_level_number'] = ((raw_level_data['energy']+1).pct_change().abs() > self.level_diff_threshold).cumsum() + 1

        # Group data with new level numbers
        self.level_data = raw_level_data.reset_index().groupby('merged_level_number').agg(
                                         energy=('energy', 'mean'), # Set energy as mean of merged levels
                                         population=('population', 'sum')) # Add the populations of merged levels
        
        self.level_mapping = raw_level_data.merged_level_number
        
        self.merged_num_shells = self.level_data.index.max()


    def display(self):
        ### Create figure and set metadata
        fig = go.Figure()

        fig.update_layout(
            title=f"Grotrian Diagram for {self.atom_name} {int_to_roman(self.ion_number)}",
            title_x=0.5,
            yaxis_title="Energy Level (eV)",
            plot_bgcolor='white',
            autosize=False,
            width=700,
            height=700,
            xaxis=dict(showticklabels=False, showgrid=False,),  # Hide x-axis labels
            yaxis=dict(showgrid=False,),  # Adjust y-axis range accordingly,
            showlegend=False,
        )

        ### Create energy level platforms in the figure
        self.level_data['centered_log_population'] = np.log10(self.level_data.population) - np.log10(np.min(self.level_data.population))
        for level_number, level_info in self.level_data.iterrows():
            level_width = level_info.centered_log_population * self.level_width_scale + self.level_width_offset
            fig.add_trace(go.Scatter(
                x=[0, 1],
                y=[level_info.energy, level_info.energy],
                mode='lines+text',
                hovertemplate='Energy: %{y:.1f}eV<br>' + f'Population: {level_info.population:.2f}' + '<extra></extra>',
                line=dict(color='black', width=level_width),
                showlegend=False))
            
            fig.add_annotation(x=1.05, y=level_info.energy, text=f'n={level_number}', showarrow=False)

        ### Create transition lines
        wavelength_range = np.log(self.max_wavelength / self.min_wavelength)
        # Plot excitation transitions
        for _, line_info in self.excit_lines.iterrows():
            lower, upper = line_info.merged_level_number_lower, line_info.merged_level_number_upper
            wavelength, num_electrons = line_info.wavelength, line_info.standard_log_num_electrons
            energy_lower, energy_upper = self.level_data.loc[lower].energy, self.level_data.loc[upper].energy

            x_end = (upper - lower) / (self.merged_num_shells - lower)

            color = matplotlib.colors.rgb2hex(self.cmap((np.log10(wavelength / self.min_wavelength)) / wavelength_range)[:3])

            # Add arrowhead
            fig.add_annotation(
                x=x_end, y=energy_upper,  # Start of arrow
                ax=0, ay=energy_lower,  # End of arrow
                xref='x', yref='y',
                axref='x', ayref='y',
                showarrow=True,
                arrowcolor=color,
                arrowhead=2,  # Arrow style
                arrowsize=1,  # Make the arrowhead 2 times bigger
                arrowwidth=num_electrons * self.transition_width_scale + self.transition_width_offset,  # Adjust width accordingly
            )

        # Plot deexcitation transitions
        for _, line_info in self.deexcit_lines.iterrows():
            lower, upper = line_info.merged_level_number_lower, line_info.merged_level_number_upper
            wavelength, num_electrons = line_info.wavelength, line_info.standard_log_num_electrons
            energy_lower, energy_upper = self.level_data.loc[lower].energy, self.level_data.loc[upper].energy

            x_end = (upper - lower) / (self.merged_num_shells - lower)

            color = matplotlib.colors.rgb2hex(cmap((np.log10(wavelength / self.min_wavelength)) / wavelength_range)[:3])

            # Add arrowhead
            fig.add_annotation(
                x=x_end, y=energy_lower,  # Start of arrow
                ax=0, ay=energy_upper,  # End of arrow
                xref='x', yref='y',
                axref='x', ayref='y',
                showarrow=True,
                arrowcolor=color,
                arrowhead=2,  # Arrow style
                arrowsize=1,  # Make the arrowhead 2 times bigger
                arrowwidth=num_electrons * self.transition_width_scale + self.transition_width_offset,  # Adjust width accordingly
            )
        
        # Add a dummy Scatter trace to display colorbar
        tickvalues = np.geomspace(self.min_wavelength, self.max_wavelength, 5)
        ticktext = [f'{val:.1e}' for val in tickvalues] 
        fig.add_trace(go.Scatter(x=[None],
                             y=[None],
                             mode='markers',
                             marker=dict(
                                 colorscale=self.colorscale, 
                                 showscale=True,
                                 cmin=np.log10(self.min_wavelength),
                                 cmax=np.log10(self.max_wavelength),
                                 colorbar=dict(title=f"Wavelength ({u.Angstrom})", thickness=5,
                                               tickvals=np.log10(tickvalues),
                                               ticktext=ticktext,
                                               outlinewidth=0, x=1.2)
                             ),
                             hoverinfo='none'
                            ))

        fig.show()
        

In [235]:
diag = GrotrianWidget(sim, atomic_number=2, ion_number=0)
diag.display()

/var/folders/y3/003r_9sn0ysgqcll3s5phtyc0000gn/T/ipykernel_2688/3738913648.py:28: PerformanceWarning:

indexing past lexsort depth may impact performance.



In [236]:
diag = GrotrianWidget(sim, atomic_number=8, ion_number=0)
diag.display()

/var/folders/y3/003r_9sn0ysgqcll3s5phtyc0000gn/T/ipykernel_2688/3738913648.py:28: PerformanceWarning:

indexing past lexsort depth may impact performance.



In [ ]:
# When number of electrons are very less (see atomic number 16),
# then just render an empty Grotrian diagram (or don't show it in options)

